## What is Loihi Graph Search?

The Loihi graph-search algorithm encodes a weighted directed graph as a spiking neural network. A wavefront launches from the destination and propagates backward along edges with delays proportional to edge costs. Whenever two wavefronts collide at a branching node, backward edges with longer arrival times are pruned, leaving only the edges along at least one shortest path. Reading out the remaining forward edges therefore yields an optimal route from source to destination.

## Setup

Import the necessary Fugu components and describe a small deterministic graph so results stay reproducible.

In [1]:
import pandas as pd
from fugu.scaffold.scaffold import Scaffold
from fugu.bricks.loihi_gs_brick import LoihiGSBrick
from fugu.backends.gsearch_backend import gsearch_Backend

# Weighted adjacency list: node -> list of (neighbor, cost)
adj = {
    0: [(1, 4), (2, 2)],
    1: [(3, 3), (4, 6)],
    2: [(1, 1), (3, 5)],
    3: [(4, 2), (5, 4)],
    4: [(5, 1)],
    5: []
}

source_node = 0
destination_node = 5

print("Adjacency list (costs):")
for node, edges in adj.items():
    formatted = ', '.join([f'{nbr} (cost {cost})' for nbr, cost in edges]) if edges else 'none'
    print(f'  {node} -> {formatted}')

Adjacency list (costs):
  0 -> 1 (cost 4), 2 (cost 2)
  1 -> 3 (cost 3), 4 (cost 6)
  2 -> 1 (cost 1), 3 (cost 5)
  3 -> 4 (cost 2), 5 (cost 4)
  4 -> 5 (cost 1)
  5 -> none


## Ground Truth Shortest Path

Compute the optimal path and cost using a standard Dijkstra implementation so we have a reference for the neuromorphic run.

In [2]:
import heapq

def dijkstra(adj, src, dst):
    dist = {src: 0}
    prev = {}
    pq = [(0, src)]
    while pq:
        d, u = heapq.heappop(pq)
        if d != dist[u]:
            continue
        if u == dst:
            break
        for v, c in adj.get(u, []):
            nd = d + c
            if nd < dist.get(v, float('inf')):
                dist[v] = nd
                prev[v] = u
                heapq.heappush(pq, (nd, v))
    path = []
    if dst in dist:
        cur = dst
        while cur != src:
            path.append(cur)
            cur = prev[cur]
        path.append(src)
        path.reverse()
    return path, dist.get(dst, float('inf'))

dijkstra_path, dijkstra_cost = dijkstra(adj, source_node, destination_node)
print(f'Ground-truth path: {dijkstra_path}')
print(f'Ground-truth cost: {dijkstra_cost}')

Ground-truth path: [0, 2, 1, 3, 4, 5]
Ground-truth cost: 9


## Build the Loihi Graph

Use `LoihiGSBrick` to convert the adjacency list into neurons and synapses that satisfy the fan-out constraints described in the Loihi paper.

In [3]:
scaffold = Scaffold()
brick = LoihiGSBrick(adj, source=source_node, destination=destination_node, name='LoihiGSTutorial')
scaffold.add_brick(brick, output=True)
scaffold.lay_bricks()

print(f'Constructed graph with {len(brick.node_list)} logical nodes (including any auxiliaries).')
print(f'Neurons in scaffold: {scaffold.graph.number_of_nodes()}')
print(f'Synapses in scaffold: {scaffold.graph.number_of_edges()}')

Constructed graph with 13 logical nodes (including any auxiliaries).
Neurons in scaffold: 13
Synapses in scaffold: 32


## Compile and Run the `gsearch_Backend`

Compile the scaffold to the Loihi graph-search backend and run the wavefront simulation until the source neuron spikes or the step budget is reached.

In [4]:
backend = gsearch_Backend()
backend.compile(scaffold, {})
backend_result = backend.run()

print(f"Backend steps executed: {backend_result['steps']}")
print(f"Source spiked: {backend_result['source_spiked']}")

# Map neuron names back to original graph nodes (skip auxiliary identifiers)
neuron_to_node = {v: k for k, v in brick.node_to_neuron.items()}
backend_path_nodes = []
for neuron_name in backend_result['path']:
    mapped = neuron_to_node.get(neuron_name)
    if isinstance(mapped, int):
        backend_path_nodes.append(mapped)

print(f"Reconstructed backend path (graph nodes): {backend_path_nodes}")

Backend steps executed: 11
Source spiked: True
Reconstructed backend path (graph nodes): [0, 2, 1, 3, 4, 5]


## Validate Path Cost

Confirm that the backend path connects the desired nodes and that its cumulative cost matches the Dijkstra baseline.

In [5]:
def edge_cost(u, v):
    for nbr, cost in adj.get(u, []):
        if nbr == v:
            return cost
    return None

backend_path_cost = 0
edge_costs = [0]
for u, v in zip(backend_path_nodes[:-1], backend_path_nodes[1:]):
    c = edge_cost(u, v)
    if c is None:
        raise RuntimeError(f"Edge ({u}->{v}) missing from adjacency list.")
    backend_path_cost += c
    edge_costs.append(c)

cost_match = backend_path_cost == dijkstra_cost
paths_match = backend_path_nodes == dijkstra_path

print(f"Backend path cost: {backend_path_cost}")
print(f"Cost parity with Dijkstra: {cost_match}")
print(f"Path equality with Dijkstra: {paths_match}")

Backend path cost: 9
Cost parity with Dijkstra: True
Path equality with Dijkstra: True


## Results Summary

Summarize the recovered path and diagnostic metrics so you can quickly confirm parity with the classical solver.

In [6]:
path_df = pd.DataFrame({
    'Step': list(range(len(backend_path_nodes))),
    'Node': backend_path_nodes,
    'Edge Cost from Prev': edge_costs
})

metrics_df = pd.DataFrame({
    'Metric': [
        'Dijkstra cost',
        'Backend cost',
        'Cost parity',
        'Paths identical',
        'Backend steps executed',
        'Source spiked'
    ],
    'Value': [
        dijkstra_cost,
        backend_path_cost,
        cost_match,
        paths_match,
        backend_result['steps'],
        backend_result['source_spiked']
    ]
})

print('=' * 72)
print('PATH DETAILS')
print('=' * 72)
print(path_df.to_string(index=False))

print('\n' + '=' * 72)
print('SUMMARY METRICS')
print('=' * 72)
print(metrics_df.to_string(index=False))

PATH DETAILS
 Step  Node  Edge Cost from Prev
    0     0                    0
    1     2                    2
    2     1                    1
    3     3                    3
    4     4                    2
    5     5                    1

SUMMARY METRICS
                Metric Value
         Dijkstra cost     9
          Backend cost     9
           Cost parity  True
       Paths identical  True
Backend steps executed    11
         Source spiked  True
